In [21]:
#3 pointer is worth 3, 
#2 point is worth 2, 
#FT is worth 1, 
#Rebounds are worth 1.2, 
#Assists are worth 1.5, 
#Blocks are worth 2, 
#Steals are worth 2, 
#Turnovers are worth -1

import pandas as pd 

#taking in the pic16b_project.csv file and giving back a new csv that has the modified fantasy stats conversion
def fantasy_stats(nba_df):
    columns = ['Year', 'PLAYER', 'GP', 'FGM', 'FG3M','FTM', 'REB', 'AST', 'STL', 'BLK', 'TOV']
    
    
    nba_df['FTM'] = nba_df['FTM'] * 1
    nba_df['FGM'] = nba_df['FGM'] * 2
    nba_df['FG3M'] = nba_df['FG3M'] * 3
    nba_df['REB'] = nba_df['REB'] * 1.2 
    nba_df['AST'] = nba_df['AST'] * 1.5  
    nba_df['STL'] = nba_df['STL'] * 2  
    nba_df['BLK'] = nba_df['BLK'] * 2  
    nba_df['TOV'] = nba_df['TOV'] * -1  
    
    # Calculate total fantasy points
    nba_df['Fantasy Points'] = nba_df['GP'] * (
        nba_df['FTM'] +
        nba_df['FG3M'] +
        nba_df['FGM'] +
        nba_df['REB'] + 
        nba_df['AST'] + 
        nba_df['STL'] + 
        nba_df['BLK'] + 
        nba_df['TOV'] 
    )
    
    # Sort the DataFrame by Fantasy Points in descending order
    nba_df = nba_df.sort_values(by='Fantasy Points', ascending=False)
    
    return nba_df[['Year', 'PLAYER','GP', 'FGM','FG3M','FTM', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'Fantasy Points']]


In [22]:
nba_api_df = pd.read_csv('pic16b_project.csv')
sorted_fantasy_df = fantasy_stats(nba_api_df)

In [23]:
print(sorted_fantasy_df)

         Year                  PLAYER  GP   FGM  FG3M   FTM    REB    AST  \
0     2018-19            James Harden  78  21.6  14.4   9.7   7.92  11.25   
1229  2023-24             Luka Dončić  70  23.0  12.3   6.8  11.04  14.70   
1238  2023-24            Nikola Jokić  79  20.8   3.3   4.5  14.88  13.50   
761   2021-22            Nikola Jokić  74  20.6   3.9   5.1  16.56  11.85   
258   2019-20            James Harden  68  19.8  13.2  10.2   7.92  11.25   
...       ...                     ...  ..   ...   ...   ...    ...    ...   
754   2020-21  Thanasis Antetokounmpo  57   2.4   0.3   0.4   2.64   1.20   
517   2019-20            Goga Bitadze  54   2.6   0.3   0.4   2.40   0.60   
1226  2022-23            Anthony Gill  59   2.4   0.3   0.8   2.04   0.90   
755   2020-21                Miye Oni  54   1.2   1.8   0.1   1.92   0.75   
257   2018-19     Sindarius Thornwell  64   0.6   0.0   0.4   0.84   0.45   

      STL  BLK  TOV  Fantasy Points  
0     4.0  1.4 -5.0         5091.06  

In [65]:
from dash import Dash, html, dcc, callback, Output, Input
import pandas as pd
import plotly.express as px

#setting up our data 
df = sorted_fantasy_df

#initialize the website
app = Dash()

#create the layout
app.layout = html.Div([
    html.H1("Basketball Fantasy Stats", style={'text-align': 'center'}),
    html.Hr(),
    dcc.Input(
        id='player-name-input',
        type='text',
        value='James Harden',
        placeholder='Enter Player Name',
        style={'margin-bottom': '20px'}
    ),
    dcc.Graph(figure={}, id='player-fantasy-graph')
    ])
        #this should be turned into a function so that it doesn't have to be repeated multiple times 
        
# Callback to update the graph
@app.callback(
    Output(component_id='player-fantasy-graph', component_property='figure'),
    Input(component_id='player-name-input', component_property='value')
)
#Making the graph
def update_graph(player_name):
        # Filter the DataFrame for the selected player
        filtered_df = df[df['PLAYER'].str.contains(player_name, case=False, na=False)]
        # Create a bar chart for the player's fantasy stats over the years
        filtered_df = filtered_df.sort_values(by='Year')
        fig = px.bar(
            filtered_df,
            x='Year',
            y='Fantasy Points',
            title=f"{player_name}'s Fantasy Points Over the Years"
        )

        return fig

# Run the Dash app
if __name__ == '__main__':
    app.run_server(mode = 'inline')

In [4]:
from dash import Dash, html, dcc, callback, Output, Input
import pandas as pd
import plotly.express as px

#setting up our data 
df = sorted_fantasy_df

#initialize the website
app = Dash()

#create the layout
app.layout = html.Div([
    html.H1("Basketball Fantasy Stats", style={'text-align': 'center'}),
    html.Hr(),
    dcc.RadioItems(
#         id='mode-selector',
#         options=[
#             {'label': 'Single Player', 'value': 'single'},
#             {'label': 'Compare Two Players', 'value': 'compare'}
#         ],
#         value='single',
#         style={'margin-bottom': '20px'}
#     ),
        options=['Single Player', 'Compare Players'],
        value='Compare Players',
        id='mode-selector'
    ),
#     dcc.Input(
#         id='player-name-input',
#         type='text',
#         value='James Harden',
#         placeholder='Enter Player Name',
#         style={'margin-bottom': '10px'}
#     ),
#     dcc.Input(
#         id='second-player-name-input',
#         type='text',
#         value='',
#         placeholder='Enter Second Player Name for Comparison',
#         style={'margin-bottom': '20px'}
#     ), 
    html.Div([
        html.Label("Player Name:"),
        dcc.Input(id='player-name-input', value='DeMar DeRozan', type='text'),
        html.Label("Second Player Name:"),
        dcc.Input(id='second-player-name-input', value = 'James Harden', type='text')
    ]),
    
    dcc.Graph(figure={}, id='player-fantasy-graph')
])


@app.callback(
    [Output('player-fantasy-graph', 'figure')],
    [Input('mode-selector', 'value'),
     Input('player-name-input', 'value'),
     Input('second-player-name-input', 'value')]
)

  
def update_graph(mode, player_name, second_player_name):
    filtered_df = df.sort_values(by='Year')
    fig = px.bar(title="Enter player names to see their fantasy stats")
    if mode == 'Single Player' and player_name:
        # Filter for the single player
        filtered_df = df[df['PLAYER'].str.contains(player_name, case=False, na=False)]
        fig = px.bar(
            filtered_df,
            x='Year',
            y='Fantasy Points',
            title=f"{player_name}'s Fantasy Points Over the Years"
        )
    elif mode == 'Compare Players' and player_name and second_player_name:
        # Filter for both players
        filtered_df = df[df['PLAYER'].str.contains(player_name + '|' + second_player_name, case=False, na=False)]
        fig = px.bar(
            filtered_df,
            x='Year',
            y='Fantasy Points',
            color='PLAYER',
            barmode='group',
            title=f"Comparison of {player_name} and {second_player_name} Fantasy Points"
        )

            
    return [fig] 

# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True)

In [24]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import pandas as pd

def predict_next_fantasy_point(df):
    predictions = []
    
    # Ensure the input data is sorted by 'Year' for proper chronological order
    df = df.sort_values(by=['PLAYER', 'Year'])
    
    for player_name in df['PLAYER'].unique():
        # Filter data for the specific player
        player_data = df[df['PLAYER'] == player_name].copy()
        
        # Ensure there's enough data for training
        if len(player_data) < 2:
            continue  # Skip this player if not enough data
        
        # Create the 'NextYearPoints' column (target)
        player_data['NextYearPoints'] = player_data['Fantasy Points'].shift(-1)
        
        # Drop the last row (no target value available for it)
        player_data = player_data.dropna()
        
        # Ensure there's enough data left after dropping NaNs
        if len(player_data) < 2:
            continue
        
        # Define X (features) and y (target)
        X = player_data['Fantasy Points'].values.reshape(-1, 1)
        y = player_data['NextYearPoints'].values
        
        # Split the data into training and testing sets (80/20 split)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        
        # Initialize and train the model (simple linear regression)
        model = LinearRegression()
        model.fit(X_train, y_train)
        
        # Predict the next year's fantasy points based on the most recent data point
        last_year_points = player_data['Fantasy Points'].iloc[-1]
        next_year_prediction = model.predict([[last_year_points]])[0]
        
        # Append the prediction to the list
        predictions.append({'PLAYER': player_name, 'Pred Fantasy Points': next_year_prediction})
    
    # Create a DataFrame from the predictions and sort by predicted points in descending order
    predictions_df = pd.DataFrame(predictions)
    draft_order_df = predictions_df.sort_values(by='Pred Fantasy Points', ascending=False).reset_index(drop=True)
    
    return draft_order_df


In [25]:
draft_order_df = predict_next_fantasy_point(sorted_fantasy_df)
print(draft_order_df)

                      PLAYER  Pred Fantasy Points
0                Luka Dončić          4487.650928
1               Nikola Jokić          4024.659698
2               Jayson Tatum          4018.447294
3    Shai Gilgeous-Alexander          3915.000000
4      Giannis Antetokounmpo          3850.151138
..                       ...                  ...
248      Juancho Hernangomez           692.160000
249             Jaxson Hayes           689.500000
250           Sterling Brown           670.800000
251           Troy Brown Jr.           624.220000
252              Cody Martin           617.240000

[253 rows x 2 columns]


In [26]:
import pandas as pd

# Calculate the standard deviation of historical points for each player
std_dev_df = sorted_fantasy_df.groupby('PLAYER')['Fantasy Points'].std().reset_index()
std_dev_df.rename(columns={'Fantasy Points': 'StanDev'}, inplace=True)

# Merge predicted points with the historical data
merged_df = sorted_fantasy_df.merge(draft_order_df, on=['PLAYER'], how='left')

# Add the standard deviation to the merged data
final_df = merged_df.merge(std_dev_df, on='PLAYER', how='left')

final_df = final_df[['Year', 'PLAYER', 'GP','Fantasy Points', 'Pred Fantasy Points', 'StanDev']]

# Sort the DataFrame for readability
final_df = final_df.sort_values(by=['PLAYER', 'Year']).reset_index(drop=True)


# Display the final filtered DataFrame
print(final_df)

         Year           PLAYER  GP  Fantasy Points  Pred Fantasy Points  \
0     2022-23       AJ Griffin  72         1196.64                  NaN   
1     2018-19     Aaron Gordon  78         2677.74          2166.879648   
2     2019-20     Aaron Gordon  62         2032.98          2166.879648   
3     2021-22     Aaron Gordon  75         2169.75          2166.879648   
4     2022-23     Aaron Gordon  68         2197.76          2166.879648   
...       ...              ...  ..             ...                  ...   
1464  2018-19    Zaza Pachulia  68          777.24                  NaN   
1465  2023-24       Zeke Nnaji  58          489.52                  NaN   
1466  2021-22  Ziaire Williams  62          956.04                  NaN   
1467  2020-21  Zion Williamson  61         2555.29                  NaN   
1468  2023-24  Zion Williamson  70         2685.20                  NaN   

         StanDev  
0            NaN  
1     250.301323  
2     250.301323  
3     250.301323  
4   

The code below needs to be edited for our functionality, names, etc. If this works it should be input into the dash app 

In [32]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Load player performance data
data = final_df.copy()

# Feature Engineering
data['FantasyPointsPerGame'] = data['Fantasy Points'] / data['GP']
data['LastSeasonTotalPoints'] = data.groupby('PLAYER')['Fantasy Points'].shift(1)  # Shift to get the previous season
data['Consistency'] = data['StanDev']

# Drop rows with missing values created by shifting or incomplete data
data.dropna(subset=['FantasyPointsPerGame', 'LastSeasonTotalPoints', 'Consistency'], inplace=True)

# Prepare the feature matrix (X) and target variable (y)
if 'TradeAccepted' not in data.columns:
    # Example: Randomly generate target data for testing; replace with actual trade data if available
    import numpy as np
    data['TradeAccepted'] = np.random.choice([0, 1], size=len(data))

X = data[['FantasyPointsPerGame', 'LastSeasonTotalPoints', 'Consistency']]
y = data['TradeAccepted']

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest Classifier
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Evaluate model performance
y_pred = model.predict(X_test)
print("Model accuracy:", accuracy_score(y_test, y_pred))


Model accuracy: 0.4946236559139785


In [33]:
def analyze_trade(player1, player2, data, model):
    
#     Analyze the likelihood of a trade being accepted between two players.
    
#     Args:
#         player1 (str): Name of the first player.
#         player2 (str): Name of the second player.
#         data (pd.DataFrame): DataFrame containing player statistics.
#         model (sklearn model): Trained Random Forest model.
        
#     Returns:
#         float: Likelihood of the trade being accepted (as a percentage).

    # Extract player data
    player1_data = data[data['PLAYER'] == player1]
    player2_data = data[data['PLAYER'] == player2]
    
    # Ensure both players exist in the dataset
    if player1_data.empty or player2_data.empty:
        raise ValueError("One or both players not found in the dataset.")
    
    # Extract relevant metrics
    player1_fppg = player1_data['FantasyPointsPerGame'].iloc[0]
    player1_total = player1_data['LastSeasonTotalPoints'].iloc[0]
    player1_consistency = player1_data['Consistency'].iloc[0]
    
    player2_fppg = player2_data['FantasyPointsPerGame'].iloc[0]
    player2_total = player2_data['LastSeasonTotalPoints'].iloc[0]
    player2_consistency = player2_data['Consistency'].iloc[0]
    
    # Create feature differences (player1 - player2)
    feature_vector = [
        player1_fppg - player2_fppg,
        player1_total - player2_total,
        player1_consistency - player2_consistency,
    ]
    
    # Convert to DataFrame for model input
    feature_df = pd.DataFrame([feature_vector], columns=['FantasyPointsPerGame', 'LastSeasonTotalPoints', 'Consistency'])
    
    # Predict likelihood of trade acceptance
    trade_likelihood = model.predict_proba(feature_df)[:, 1][0]
    
    return trade_likelihood * 100


In [34]:
print(analyze_trade('Luka Dončić', 'James Harden', data, model))

82.0
